In [1]:
import pandas as pd
import numpy as np
import pulp
import os
os.getcwd()
os.chdir("..")

# Process data

In [2]:
raw_workmode = pd.read_excel('InputData.xlsx', sheet_name="Mode")
raw_demand = pd.read_excel('InputData.xlsx', sheet_name="DemandV2") # ver2
workmode = {(row['Mode'].item(), row['Shift'].item()): row['Capa'].item() for _, row in raw_workmode.iterrows()}
demand = {row['Week'].item(): row['Demand'].item() for _, row in raw_demand.iterrows()} # ver2

In [3]:
WeekIndex = raw_demand.Week.tolist()
ShiftIndex = [1,2,3]
bigM = 99999

In [4]:
numS = 17
numC = 12
FC = {
    (i, j): raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].item() * (
        8.3 * 1.5 * numC if (j in [2, 3] and i == 1) else
        8.3 * 1.5 * numS if (j in [2, 3] and i == 2) else
        8.3 * numC if (j in [1] and i == 1) else
        8.3 * numS if (j in [1] and i == 2) else
        0)
    for i in [1, 2]
    for j in ShiftIndex
    if not raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].empty}

FC_sunday = {
    (i, j): raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].item() * (
        8.3 * 2 * numC if (i == 1) else
        8.3 * 2 * numS if (i == 2) else
        0)
    for i in [1, 2]
    for j in ShiftIndex
    if not raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].empty}

# Algorithm

In [5]:
# Create the model
model = pulp.LpProblem("3PL_Workforce_Allocation", pulp.LpMinimize)

# Variable
m_conso = pulp.LpVariable.dicts("m_conso", (ShiftIndex, WeekIndex), lowBound=0, cat='Integer')
m_sepa = pulp.LpVariable.dicts("m_sepa", (ShiftIndex, WeekIndex), lowBound=0, cat='Integer')
# total_shifts = pulp.LpVariable.dicts("total_shifts", WeekIndex, cat='Integer')
x = pulp.LpVariable.dicts("x", WeekIndex, cat='Binary')
delta_conso = pulp.LpVariable.dicts("delta_conso", (ShiftIndex, WeekIndex), lowBound=0, upBound=1, cat='Integer')
delta_sepa = pulp.LpVariable.dicts("delta_sepa", (ShiftIndex, WeekIndex), lowBound=0, upBound=1, cat='Integer')

# Objective function: 
model += pulp.lpSum(FC[1, sh] * m_conso[sh][w] + FC_sunday[1, sh]*delta_conso[sh][w] for sh in ShiftIndex for w in WeekIndex) + \
         pulp.lpSum(FC[2, sh] * m_sepa[sh][w] + FC_sunday[2, sh]*delta_sepa[sh][w] for sh in ShiftIndex for w in WeekIndex)

# Constraint

# Can't work more than 3 shifts 3 per week
for w in WeekIndex:
    model += m_conso[3][w] + m_sepa[3][w] <= 3

# Satisfy demand ver 2
for w in WeekIndex:
    model += pulp.lpSum(workmode[1, sh]*m_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(workmode[2, sh]*m_sepa[sh][w] for sh in ShiftIndex) >= demand[w]

# Min 5 days a week
for w in WeekIndex:
    model += pulp.lpSum(m_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(m_sepa[sh][w] for sh in ShiftIndex) >= 5

# Max 7 days a week
for w in WeekIndex:
    model += pulp.lpSum(m_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(m_sepa[sh][w] for sh in ShiftIndex) <= 7

# If sum = 7, then apply 2 times wages
## If the total of m_conso and m_sepa is 7, activate x[w]
for w in WeekIndex:
    total_shifts = pulp.lpSum(m_conso[sh][w] + m_sepa[sh][w] for sh in ShiftIndex)
    model += total_shifts - 7 <= bigM * (1 - x[w])
    model += total_shifts - 7 >= -bigM * (1 - x[w])

    model += total_shifts <= 6 + bigM * x[w]

## If x[w] is 1, then either delta_conso or delta_sepa must increase by exactly 1
for w in WeekIndex:
    model += pulp.lpSum(delta_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(delta_sepa[sh][w] for sh in ShiftIndex) == x[w]

## Only add to available mode
for w in WeekIndex:
    for sh in ShiftIndex:
        model += m_conso[sh][w] >= delta_conso[sh][w] 
for w in WeekIndex:
    for sh in ShiftIndex:
        model += m_sepa[sh][w] >= delta_sepa[sh][w]

# If z[w] is 0, then delta_conso and delta_sepa must both be 0 for all shifts
for w in WeekIndex:
    for sh in ShiftIndex:
        model += delta_conso[sh][w] <= x[w]
        model += delta_sepa[sh][w] <= x[w]

# Solve the problem using COIN_CMD without the tol argument
solver = pulp.PULP_CBC_CMD(gapRel=0.00001)  # Built-in solver
model.solve(solver)


results_conso = []
for sh in ShiftIndex:
    for w in WeekIndex:
        results_conso.append({
            'shift': sh,
            'week': w,
            'choose': m_conso[sh][w].varValue
        })

res_delta_conso = []
for sh in ShiftIndex:
    for w in WeekIndex:
        res_delta_conso.append({
                'shift': sh,
                'week': w,
                'choose': delta_conso[sh][w].varValue
            })


results_sepa = []
for sh in ShiftIndex:
    for w in WeekIndex:
        results_sepa.append({
            'shift': sh,
            'week': w,
            'choose': m_sepa[sh][w].varValue
        })

res_delta_sepa = []
for sh in ShiftIndex:
    for w in WeekIndex:
        res_delta_sepa.append({
                'shift': sh,
                'week': w,
                'choose': delta_sepa[sh][w].varValue
            })

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/lamjackie/miniconda3/envs/ych-wf-allocation/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/a64aebca5e68435bb7f28714b07a3504-pulp.mps -ratio 1e-05 -timeMode elapsed -branch -printingOptions all -solution /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/a64aebca5e68435bb7f28714b07a3504-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 165 COLUMNS
At line 1046 RHS
At line 1207 BOUNDS
At line 1312 ENDATA
Problem MODEL has 160 rows, 104 columns and 576 elements
Coin0008I MODEL read with 0 errors
ratioGap was changed from 0 to 1e-05
Option for timeMode changed from cpu to elapsed
Continuous objective value is 38765.7 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 40 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 48 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tig

In [6]:
df_results_conso=pd.DataFrame(results_conso)
df_results_conso['mode'] = "c"
df_results_sepa = pd.DataFrame(results_sepa)
df_results_sepa['mode'] = "s"
result = pd.concat([df_results_sepa, df_results_conso])
result_pivot=pd.DataFrame(result.pivot_table(index=["mode", "shift"], columns=["week"], values=["choose"], aggfunc="sum").reset_index(drop=False))
result_pivot

mode shift choose                                   
week                 1    2    3    4    5    6    7    8
0       c     1    1.0  0.0  1.0  4.0  3.0  4.0  5.0  5.0
1       c     2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2       c     3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3       s     1    4.0  6.0  5.0  1.0  2.0  1.0  0.0  0.0
4       s     2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5       s     3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [7]:
with pd.ExcelWriter("result_v3.xlsx") as writer:
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    result_pivot.to_excel(writer, sheet_name="Pivot")
    result.to_excel(writer, sheet_name="Results", index=False)
    raw_workmode.to_excel(writer, sheet_name="Workforce", index=False)
    raw_demand.to_excel(writer, sheet_name="Demand", index=False)

# Validate

In [ ]:
for w in WeekIndex:
    total_shifts_value = sum(m_conso[sh][w].varValue + m_sepa[sh][w].varValue for sh in ShiftIndex)
    print(f"Total shifts for week {w}: {total_shifts_value}")

In [ ]:
# Calculate and print total shifts for each week 46313.9 
for w in WeekIndex:
    total_shifts_value = sum(m_sepa[sh][w].varValue + delta_sepa[sh][w].varValue for sh in ShiftIndex)
    print(f"Total shifts for week {w}: {total_shifts_value}")

In [ ]:
for w in WeekIndex:
    print(f"Total shifts for week {w}: {x[w].varValue}")

In [ ]:
res_delta_sepa

In [ ]:
res_delta_conso

In [ ]:
workmode

In [ ]:
raw_demand